In [1]:
import Pkg
Pkg.activate("../../tensor")
import tensor as ten
using ITensors

  Activating project at `c:\Users\kakin\home\workspace\tensor`


In [2]:
import LinearAlgebra as LA

In [ ]:
#関数修正済み

In [3]:
#Index
#大きな足
α = Index(2, "α")
ξ = Index(2, "ξ")
β = Index(2, "β")
η = Index(2, "η")

#小さな足
i = Index(2, "i")
j = Index(2, "j")
k = Index(2, "k")
l = Index(2, "l")

#つぶす足
c1 = Index(2 , "c1")
c2 = Index(2 , "c2")

#定数K
K_const=-0.1

-0.1

In [4]:
#ここがおかしい
#4脚テンソルの定義
W = ten.four_leg_tensor_def(i, j, k, l,K_const)
#2脚テンソルの定義
C = ten.two_leg_tensor_def(α, β, c1, c2, K_const)
#3脚テンソルの定義
PR = ten.three_leg_tensor_def(α, ξ, l, c1, K_const)
#3脚テンソルの定義
PC = ten.three_leg_tensor_def(β, η , k, c1, K_const)

ITensor ord=3 (dim=2|id=465|"β") (dim=2|id=846|"η") (dim=2|id=924|"k")
NDTensors.Dense{Float64, Vector{Float64}}

In [6]:
function Expand_PR(PR::ITensor,W::ITensor,α::Index{Int64},k::Index{Int64},ξ::Index{Int64},i::Index{Int64},l::Index{Int64})
    #拡大
    PR2=PR*W

    #文字の結合
    αk = combiner(α, k)
    ξi = combiner(ξ, i)

    PR2_ext = αk * PR2
    ξ = Index(dim(uniqueinds(αk)[1]), "ξ")
    replaceinds!(PR2_ext, uniqueinds(αk)[1] => ξ)

    PR2_ext = PR2_ext * ξi 
    α = Index(dim(uniqueinds(ξi)[1]), "α")
    replaceinds!(PR2_ext, uniqueinds(ξi)[1] => α)

    replaceinds!(PR2_ext, uniqueinds(j)[1] => l)

    return PR2_ext,ξ,α,ξi
end

PR2,ξ,α,ξi=Expand_PR(PR,W,α,k,ξ,i,l)


(ITensor ord=3
Dim 1: (dim=4|id=45|"ξ")
Dim 2: (dim=2|id=247|"l")
Dim 3: (dim=4|id=289|"α")
NDTensors.Dense{Float64, Vector{Float64}}
 4×2×4
[:, :, 1] =
 3.1196490101528607  4.65396944131818
 3.832464789712549   5.717365626133738
 3.6703200460356395  3.6703200460356395
 4.49182469764127    4.49182469764127

[:, :, 2] =
 3.6703200460356395  5.475474092923811
 3.0109601381069178  4.49182469764127
 4.49182469764127    4.49182469764127
 3.6703200460356395  3.6703200460356395

[:, :, 3] =
 3.6703200460356395  3.6703200460356395
 4.49182469764127    4.49182469764127
 4.49182469764127    3.0109601381069178
 5.475474092923811   3.6703200460356395

[:, :, 4] =
 4.49182469764127    4.49182469764127
 3.6703200460356395  3.6703200460356395
 5.717365626133738   3.832464789712549
 4.65396944131818    3.119649010152861, (dim=4|id=45|"ξ"), (dim=4|id=289|"α"), ITensor ord=3
Dim 1: (dim=4|id=827|"CMB,Link")
Dim 2: (dim=2|id=860|"ξ")
Dim 3: (dim=2|id=79|"i")
NDTensors.Combiner
 4×2×2

Permutation of bloc

In [7]:
function Expand_PC(PC::ITensor,W::ITensor,β::Index{Int64},l::Index{Int64},η::Index{Int64},j::Index{Int64},i::Index{Int64})
    PC2=PC*W
    βl = combiner(β, l)
    ηj = combiner(η, j)

    PC2_ext = βl * PC2
    η = Index(dim(uniqueinds(βl)[1]), "η")
    replaceinds!(PC2_ext, uniqueinds(βl)[1] => η)

    PC2_ext = PC2_ext * ηj 
    β = Index(dim(uniqueinds(ηj)[1]), "β")
    replaceinds!(PC2_ext, uniqueinds(ηj)[1] => β)

    replaceinds!(PC2_ext, uniqueinds(k)[1] => i)

    return PC2_ext,η,β,ηj
end

PC2,η,β,ηj=Expand_PR(PC,W,β,l,η,j,i)

(ITensor ord=3
Dim 1: (dim=4|id=162|"ξ")
Dim 2: (dim=2|id=79|"i")
Dim 3: (dim=4|id=391|"α")
NDTensors.Dense{Float64, Vector{Float64}}
 4×2×4
[:, :, 1] =
 3.1196490101528607  4.65396944131818
 3.832464789712549   5.717365626133738
 3.6703200460356395  3.6703200460356395
 4.49182469764127    4.49182469764127

[:, :, 2] =
 3.6703200460356395  5.475474092923811
 3.0109601381069178  4.49182469764127
 4.49182469764127    4.49182469764127
 3.6703200460356395  3.6703200460356395

[:, :, 3] =
 3.6703200460356395  3.6703200460356395
 4.49182469764127    4.49182469764127
 4.49182469764127    3.0109601381069178
 5.475474092923811   3.6703200460356395

[:, :, 4] =
 4.49182469764127    4.49182469764127
 3.6703200460356395  3.6703200460356395
 5.717365626133738   3.832464789712549
 4.65396944131818    3.119649010152861, (dim=4|id=162|"ξ"), (dim=4|id=391|"α"), ITensor ord=3
Dim 1: (dim=4|id=32|"CMB,Link")
Dim 2: (dim=2|id=846|"η")
Dim 3: (dim=2|id=99|"j")
NDTensors.Combiner
 4×2×2

Permutation of bloc

In [32]:
typeof(ξi)

ITensor

In [34]:
function Expand_C(PR::ITensor,C::ITensor,PC::ITensor,W::ITensor,ξi::ITensor,ηj::ITensor)
    #拡大
    C2=PR*C*PC*W

    #文字の結合
    C2_ext = ξi * C2 * ηj

    replaceinds!(C2_ext, uniqueinds(ξi)[1] => α)
    replaceinds!(C2_ext, uniqueinds(ηj)[1] => β)

    return C2_ext
end

C2=Expand_C(PR,C,PC,W,ξi,ηj)

ITensor ord=2 (dim=4|id=289|"α") (dim=4|id=391|"α")
NDTensors.Dense{Float64, Vector{Float64}}

In [9]:
PR2|>display
PC2|>display
C2|>display
W|>display

ITensor ord=3 (dim=4|id=45|"ξ") (dim=2|id=247|"l") (dim=4|id=289|"α")
NDTensors.Dense{Float64, Vector{Float64}}

ITensor ord=3 (dim=4|id=162|"ξ") (dim=2|id=79|"i") (dim=4|id=391|"α")
NDTensors.Dense{Float64, Vector{Float64}}

ITensor ord=2 (dim=4|id=289|"α") (dim=4|id=391|"α")
NDTensors.Dense{Float64, Vector{Float64}}

ITensor ord=4 (dim=2|id=79|"i") (dim=2|id=99|"j") (dim=2|id=924|"k") (dim=2|id=247|"l")
NDTensors.Dense{Float64, Vector{Float64}}

In [10]:
C3=PR2*PC2*C2*W

ITensor ord=4 (dim=4|id=45|"ξ") (dim=4|id=162|"ξ") (dim=2|id=99|"j") (dim=2|id=924|"k")
NDTensors.Dense{Float64, Vector{Float64}}

In [11]:
# 固有値分解
D, U = eigen(C2, α, β)

# インデックスの取り出し
dl = uniqueind(D, U)
dr = commonind(D, U)

# 正しい方法でUlを作成（置き換えは個別に！）
Ul = replaceinds(U, (β => α, dr => dl))

# 誤差チェック
diff = norm(C2 * U - Ul * D)
println("norm(C2 * U - Ul * D) = ", diff)
println("一致する？ ", diff < 1e-11)



norm(C2 * U - Ul * D) = 1.1791567914286052e-12
一致する？ true


In [18]:
Ul*D*U

ITensor ord=2 (dim=4|id=289|"α") (dim=4|id=391|"α")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [19]:
resUl

ITensor ord=2 (dim=3|id=921|"χ")' (dim=4|id=289|"α")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [20]:
resD

ITensor ord=2 (dim=3|id=921|"χ")' (dim=3|id=921|"χ")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [21]:
size(resU)|>display
size(resUl)|>display
size(resD)|>display

(4, 3)

(3, 4)

(3, 3)

In [22]:
A=resUl*resD*resU
#テスト

ITensor ord=2 (dim=4|id=289|"α") (dim=4|id=391|"α")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [23]:
C2

ITensor ord=2 (dim=4|id=289|"α") (dim=4|id=391|"α")
NDTensors.Dense{Float64, Vector{Float64}}

In [24]:
α

(dim=4|id=289|"α")

In [25]:
PR

ITensor ord=3 (dim=2|id=870|"α") (dim=2|id=860|"ξ") (dim=2|id=247|"l")
NDTensors.Dense{Float64, Vector{Float64}}

In [26]:
PR2=PR*W

ITensor ord=5 (dim=2|id=870|"α") (dim=2|id=860|"ξ") (dim=2|id=79|"i") (dim=2|id=99|"j") (dim=2|id=924|"k")
NDTensors.Dense{Float64, Vector{Float64}}

In [27]:
α

(dim=4|id=289|"α")

In [28]:
#=using ITensors

# 対角化（例: A is ITensor with indices α, β）
D, U = eigen(C2, α, β)

dr = commonind(D, U)
dl = uniqueind(D, U)

χ = 2

# ✅ 新しい次元χのIndexを新規生成（タグは同じでもOK）
drχ = Index(χ, "Link,eigen")
dlχ = prime(drχ)

# ✅ Uのインデックス置き換え（単一の置換ならこれで十分）
Uχ = replaceind(U, dr => drχ)

# ✅ Dχ の構築
Dχ = ITensor(dlχ, drχ)
for n in 1:χ
    Dχ[dlχ => n, drχ => n] = D[dl => n, dr => n]
end

# ✅ Ulχ の構成（ここも replaceind で十分）
Ulχ = replaceind(Uχ, β => α)
Ulχ = replaceind(Ulχ, drχ => dlχ)

# ✅ 再構成して誤差確認
Aχ = Ulχ * Dχ
println("圧縮誤差: ", norm(C2 - Aχ))
=#

In [29]:
D
norm(Ul*D*U-C2)

1.1193247171101585e-12

In [30]:
A=Array(C2,α,β)
Base.print_matrix(stdout,A)

 253.49616979434984  249.95712926003972  304.66466954142385  312.7072764780771
 249.95712926003972  244.49265776061023  299.22179615334585  304.66466954142385
 304.6646695414238   299.22179615334585  244.4926577606102   249.95712926003972
 312.7072764780772   304.66466954142385  249.95712926003972  253.49616979434984